# **Project name:** Calorie estimation of various foods 



# Problem statement: 
Given a collection of food images with a calibration item thumb annotated with the food name and an unlabeled set of food images from the same category of food, identify the food and estimate the amount and calories consumed.

**Steps for building this system:**


1.   Review over existing works

*   [An Image Procesing Approach for Calorie
Intake Measurement](https://www.researchgate.net/profile/Shervin-Shirmohammadi-2/publication/241633012_An_Image_Processing_Approach_for_Calorie_Intake_Measurement/links/5ab5e7b0aca2722b97cad998/An-Image-Processing-Approach-for-Calorie-Intake-Measurement.pdf)


*   [Food calorie estimation using machine learning and image processing](https://d1wqtxts1xzle7.cloudfront.net/59165453/V5I2-193520190507-93463-zup54f-with-cover-page-v2.pdf?Expires=1647348749&Signature=aqrQzNeQwytLCh1bzlAbrecTdC23E9Gbl~xFS1rb4KIvvjpKiFMPAST5JP6gEXo~5H6GeirlEsjSkR5yFacpEKpz2AwGuaWwBkQOpA0ugZww6HCX~zmicGg82J6gnTvTw0ZtreweX4Z30Nj4HKBywwpXS5sceG~-Afwxekco2KY1iFir~290bWCa9z5q4XGT6BFSSjy6O705q6h5tqLfww1edi-XGlwHsLSEXJPj9ZCsUkvX~zb9Y5KRFQJhGwwzyZy~VLFSHq1qpmRuAZbnCP7Ju33f668LQsj8Q4jc~SrFH~l6-tTKdUWIXphRYgiESelYI60wdaL0Qfe6WtnxBw__&Key-Pair-Id=APKAJLOHF5GGSLRBV4ZA)

*   [Image Processing Based Classification of Energy Sources in Eatables Using Artificial Intelligence](https://www.annalsofrscb.ro/index.php/journal/article/download/2277/1900)


*  [Application of Deep Learning in Food: A Review](https://sci-hub.hkvisa.net/10.1111/1541-4337.12492) 



*  [Deep Learning-Based Food Calorie Estimation Method in Dietary Assessment](https://github.com/Liang-yc/CalorieEstimation/tree/master/faster_rcnn-master) 


2. Dataset: 

*   [Fruit360](https://www.kaggle.com/moltean/fruits?select=fruits-360-original-size)

*  [FooDD](https://drive.google.com/drive/folders/1rmVS_n6P1859trFxKhY7iHVywjTIRYwn) 


*   [ECUSTFD](https://github.com/Liang-yc/ECUSTFD-resized-)




3.  Classification and detection of food images via algortihms: 


*  Histogram of Oriented Gradients (HOG) Feature Descriptor

*  Region-based Convolutional Neural Networks (R-CNN) 

*  Region-based Fully Convolutional Network (R-FCN) 


*  Single Shot Detector (SSD)

*  Spatial Pyramid Pooling (SPP-net) 

*  YOLO (You Only Look Once) e.t.c. 

We are still looking for suitable algorithms for detection.


4. Now there are some challanges regarding calorie estimation. First challenge is to predict ingredients like predicting type of pizza from various type. Then to predict its weight. Now weight is calculated as, 



> **M = V.D**

Where M is mass, V is volume and D is density. Now getting volume from 2d image is very difficult. And still there is no significant progress in this field. So we opted a different approach. We will use image segmentation methods for this. To acquire the contour of the fruit and the contour of the thumb, a combination of approaches including canny edge detection, watershed segmentation, morphological operators, and Otsu's method were utilized to segment the food item. For calibrating purposes, we use the thumb finger. While taking the snapshot, the thumb is put next to the dish, giving us an indication of the real-life size of the food item and assisting in accurate volume estimation. Here is a demo of our image segmentation: 























**Importing libraries**

In [1]:
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow

**Defining function**

In [2]:
def getAreaOfFood(img1):
    data=os.path.join(os.getcwd(),"images")
    if os.path.exists(data):
        print('folder exist for images at ',data)
    else:
        os.mkdir(data)
        print('folder created for images at ',data)
        
    cv2.imwrite('{}\\1 original image.jpg'.format(data),img1)
    img = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    cv2.imwrite('{}\\2 original image BGR2GRAY.jpg'.format(data),img)
    cv2_imshow(img)
    img_filt = cv2.medianBlur( img, 5)
    cv2.imwrite('{}\\3 img_filt.jpg'.format(data),img_filt)
    cv2_imshow(img_filt)
    img_th = cv2.adaptiveThreshold(img_filt,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,21,2)
    cv2.imwrite('{}\\4 img_th.jpg'.format(data),img_th)
    cv2_imshow(img_th)
    contours, hierarchy = cv2.findContours(img_th, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) #make change here


	# find contours. sort. and find the biggest contour. the biggest contour corresponds to the plate and fruit.
    mask = np.zeros(img.shape, np.uint8)
    largest_areas = sorted(contours, key=cv2.contourArea)
    cv2.drawContours(mask, [largest_areas[-1]], 0, (255,255,255,255), -1)
    cv2.imwrite('{}\\5 mask.jpg'.format(data),mask)
    cv2_imshow(mask)
    img_bigcontour = cv2.bitwise_and(img1,img1,mask = mask)
    cv2.imwrite('{}\\6 img_bigcontour.jpg'.format(data),img_bigcontour)
    cv2_imshow(img_bigcontour)

	# convert to hsv. otsu threshold in s to remove plate
    hsv_img = cv2.cvtColor(img_bigcontour, cv2.COLOR_BGR2HSV)
    cv2.imwrite('{}\\7 hsv_img.jpg'.format(data),hsv_img)
    cv2_imshow(hsv_img)
    h,s,v = cv2.split(hsv_img)
    mask_plate = cv2.inRange(hsv_img, np.array([0,0,50]), np.array([200,90,250]))
    cv2.imwrite('{}\\8 mask_plate.jpg'.format(data),mask_plate)
    cv2_imshow(mask_plate)
    mask_not_plate = cv2.bitwise_not(mask_plate)
    cv2.imwrite('{}\\9 mask_not_plate.jpg'.format(data),mask_not_plate)
    cv2_imshow(mask_not_plate)
    fruit_skin = cv2.bitwise_and(img_bigcontour,img_bigcontour,mask = mask_not_plate)
    cv2.imwrite('{}\\10 fruit_skin.jpg'.format(data),fruit_skin)
    cv2_imshow(fruit_skin)

	#convert to hsv to detect and remove skin pixels
    hsv_img = cv2.cvtColor(fruit_skin, cv2.COLOR_BGR2HSV)
    cv2.imwrite('{}\\11 hsv_img.jpg'.format(data),hsv_img)
    cv2_imshow(hsv_img)
    skin = cv2.inRange(hsv_img, np.array([0,10,60]), np.array([10,160,255])) #Scalar(0, 10, 60), Scalar(20, 150, 255)
    cv2.imwrite('{}\\12 skin.jpg'.format(data),skin)
    cv2_imshow(skin)
    not_skin = cv2.bitwise_not(skin); #invert skin and black
    cv2.imwrite('{}\\13 not_skin.jpg'.format(data),not_skin)
    cv2_imshow(not_skin)
    fruit = cv2.bitwise_and(fruit_skin,fruit_skin,mask = not_skin) #get only fruit pixels
    cv2.imwrite('{}\\14 fruit.jpg'.format(data),fruit)
    cv2_imshow(fruit)
    
    fruit_bw = cv2.cvtColor(fruit, cv2.COLOR_BGR2GRAY)
    cv2.imwrite('{}\\15 fruit_bw.jpg'.format(data),fruit_bw)
    cv2_imshow(fruit_bw)
    fruit_bin = cv2.inRange(fruit_bw, 10, 255) #binary of fruit
    cv2.imwrite('{}\\16 fruit_bw.jpg'.format(data),fruit_bin)
    cv2_imshow(fruit_bin)

	#erode before finding contours
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    erode_fruit = cv2.erode(fruit_bin,kernel,iterations = 1)
    cv2.imwrite('{}\\17 erode_fruit.jpg'.format(data),erode_fruit)
    cv2_imshow(erode_fruit)

	#find largest contour since that will be the fruit
    img_th = cv2.adaptiveThreshold(erode_fruit,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    cv2.imwrite('{}\\18 img_th.jpg'.format(data),img_th)
    cv2_imshow(img_th)
    contours, hierarchy = cv2.findContours(img_th, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask_fruit = np.zeros(fruit_bin.shape, np.uint8)
    largest_areas = sorted(contours, key=cv2.contourArea)
    cv2.drawContours(mask_fruit, [largest_areas[-2]], 0, (255,255,255), -1)
    cv2.imwrite('{}\\19 mask_fruit.jpg'.format(data),mask_fruit)
    cv2_imshow(mask_fruit)

	#dilate now
    kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    mask_fruit2 = cv2.dilate(mask_fruit,kernel2,iterations = 1)
    cv2.imwrite('{}\\20 mask_fruit2.jpg'.format(data),mask_fruit2)
    cv2_imshow(mask_fruit2)
    fruit_final = cv2.bitwise_and(img1,img1,mask = mask_fruit2)
    cv2.imwrite('{}\\21 fruit_final.jpg'.format(data),fruit_final)
    cv2_imshow(fruit_final)
    
	#find area of fruit
    img_th = cv2.adaptiveThreshold(mask_fruit2,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    cv2.imwrite('{}\\22 img_th.jpg'.format(data),img_th)
    cv2_imshow(img_th)
    contours, hierarchy = cv2.findContours(img_th, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    largest_areas = sorted(contours, key=cv2.contourArea)
    fruit_contour = largest_areas[-2]
    fruit_area = cv2.contourArea(fruit_contour)

	
	#finding the area of skin. find area of biggest contour
    skin2 = skin - mask_fruit2
    cv2.imwrite('{}\\23 skin2.jpg'.format(data),skin2)
    cv2_imshow(skin2)
	#erode before finding contours
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    skin_e = cv2.erode(skin2,kernel,iterations = 1)
    cv2.imwrite('{}\\24 skin_e .jpg'.format(data),skin_e )
    cv2_imshow(skin_e)
    img_th = cv2.adaptiveThreshold(skin_e,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    cv2.imwrite('{}\\25 img_th.jpg'.format(data),img_th)
    cv2_imshow(img_th)
    contours, hierarchy = cv2.findContours(img_th, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask_skin = np.zeros(skin.shape, np.uint8)
    largest_areas = sorted(contours, key=cv2.contourArea)
    cv2.drawContours(mask_skin, [largest_areas[-2]], 0, (255,255,255), -1)
    cv2.imwrite('{}\\26 mask_skin.jpg'.format(data),mask_skin)
    cv2_imshow(mask_skin)
    
    
    skin_rect = cv2.minAreaRect(largest_areas[-2])
    box = cv2.boxPoints(skin_rect)
    box = np.int0(box)
    mask_skin2 = np.zeros(skin.shape, np.uint8)
    cv2.drawContours(mask_skin2,[box],0,(255,255,255), -1)
    cv2.imwrite('{}\\27 mask_skin2.jpg'.format(data),mask_skin2)
    cv2_imshow(mask_skin2)
    
    pix_height = max(skin_rect[1])
    pix_to_cm_multiplier = 5.0/pix_height
    skin_area = cv2.contourArea(box)
    
    
    return fruit_area,fruit_bin ,fruit_final,skin_area, fruit_contour, pix_to_cm_multiplier

**Connecting Google drive to import image**

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


**Testing these segementation**

In [ ]:
    import cv2
    img1 = cv2.imread(r"/content/gdrive/MyDrive/test/1 (21).jpg")
    cv2_imshow(img1)


In [ ]:
    img1 = cv2.imread(r"/content/gdrive/MyDrive/test/1 (21).jpg")
    img = cv2.resize(img1,(1000,1000))
    area, bin_fruit, img_fruit, skin_area, fruit_contour, pix_to_cm_multiplier = getAreaOfFood(img)

Output hidden; open in https://colab.research.google.com to view.